# 4. Modeling
## 4.1 Select Modeling Techniques
### Outputs:
- Modeling Technique
- Modeling Assumptions



In [257]:
from sklearn.linear_model import LogisticRegression
import nltk
import pandas as pd
import math
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
from matplotlib import gridspec

from sklearn import datasets, linear_model
import numpy as np
from numbers import Number

from sklearn.model_selection import train_test_split, cross_val_score
import sklearn.metrics as metrics

from sklearn import tree
from sklearn.ensemble import GradientBoostingRegressor, VotingClassifier

import warnings
warnings.filterwarnings('ignore')


In [275]:
train_dataframe= pd.read_csv("../data/train_dummied.csv")
train_dataframe=train_dataframe.set_index('Id')
y=np.array(train_dataframe.pop('SalePrice'))
X=np.array(train_dataframe)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20)

KeyError: 'SalePrice'

### Logistic Regression

In [259]:
lr = LogisticRegression(C=1000.0, random_state=0)
lr.fit(X_train, y_train)
y_pred_lr=lr.predict(X_test)

In [260]:
print ('Mean Squared Error:',np.sqrt(metrics.mean_squared_error(y_pred_lr,y_test)))
scores = cross_val_score(clf, X_train, y_train, cv=5)
print("Accuracy: %0.5f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

Mean Squared Error: 49805.5908092
Accuracy: 0.72980 (+/- 0.12)


###  Decision Tree Regression 

In [261]:
clf = tree.DecisionTreeRegressor()
clf = clf.fit(X_train, y_train)
y_pred_clf=clf.predict(X_test)

In [262]:
print ('Mean Squared Error:',np.sqrt(metrics.mean_squared_error(y_pred_clf,y_test)))
scores = cross_val_score(clf, X_train, y_train, cv=5)
print("Accuracy: %0.5f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

Mean Squared Error: 33225.3529899
Accuracy: 0.76537 (+/- 0.07)


## Ridge Linear Regression

In [263]:
ridge = linear_model.Ridge (alpha = .5)
ridge.fit (X_train,y_train)
y_pred_ridge = ridge.predict(X_test)

In [264]:
print ('Mean Squared Error:',np.sqrt(metrics.mean_squared_error(y_pred_ridge,y_test)))
scores = cross_val_score(ridge, X_train, y_train, cv=5)
print("Accuracy: %0.5f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

Mean Squared Error: 22510.7073937
Accuracy: 0.89722 (+/- 0.03)


## Lasso Linear Regression

In [265]:
lasso = linear_model.Lasso(alpha = 1.5)
lasso = lasso.fit(X_train,y_train)
y_pred_lasso = lasso.predict(X_test)

In [266]:
print ('Mean Squared Error:',np.sqrt(metrics.mean_squared_error(y_pred_lasso,y_test)))
scores = cross_val_score(lasso, X_train, y_train, cv=5)
print("Accuracy: %0.5f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

Mean Squared Error: 22811.4804301
Accuracy: 0.88735 (+/- 0.04)


## Elastic Net

In [267]:
ENST = linear_model.ElasticNetCV(alphas=[0.0001, 0.0005, 0.001, 0.01, 0.1, 1, 10], 
                                    l1_ratio=[.01, .1, .5, .9, .99], 
                                    max_iter=5000)
ENST = ENST.fit(X_train, y_train)
y_pred_enst = ENST.predict(X_test)

In [268]:
print ('Mean Squared Error:',np.sqrt(metrics.mean_squared_error(y_pred_enst,y_test)))
scores = cross_val_score(ENST, X_train, y_train, cv=5)
print("Accuracy: %0.5f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

Mean Squared Error: 20408.8095899
Accuracy: 0.90297 (+/- 0.03)


## Gradient Tree Boosting

In [269]:
gtb = GradientBoostingRegressor(n_estimators=100, 
                                learning_rate=0.1,
                                max_depth=1,
                                random_state=0, 
                                loss='ls')
gtb = gtb.fit(X_train,y_train)
y_pred_gtb = gtb.predict(X_test)

In [270]:
print ('Mean Squared Error:',np.sqrt(metrics.mean_squared_error(y_pred_gtb,y_test)))
scores = cross_val_score(gtb, X_train, y_train, cv=5)
print("Accuracy: %0.5f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

Mean Squared Error: 27308.4977026
Accuracy: 0.87753 (+/- 0.04)


## Voting Classifier

In [271]:
results=pd.DataFrame({'Ridge':y_pred_ridge,
                     'Lasso':y_pred_lasso,
                     'ENST':y_pred_enst,
                     'test':y_test})

In [272]:
stacker= linear_model.LinearRegression()
stacker.fit(results[['Ridge', 'Lasso', 'ENST']], results['test'])

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

In [273]:
scores = cross_val_score(stacker, results[['Ridge', 'Lasso', 'ENST']], results['test'], cv=5)
print("Accuracy: %0.5f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

Accuracy: 0.92059 (+/- 0.02)


In [274]:
results['stacked']=stacker.predict(results[['Ridge', 'Lasso', 'ENST']])
results

,ENST,Lasso,Ridge,test,stacked
0,139793.370683,139455.830522,138663.267430,142953.0,142626.498877
1,367907.937438,379347.657935,377585.526559,310000.0,357473.075624
2,187694.539601,184974.915252,184743.553049,183500.0,188087.799309
3,221691.116244,227883.602397,223090.012515,187000.0,223309.475145
4,180810.850305,185498.656711,185363.549014,164500.0,179595.453811
5,372207.455049,385625.696555,384563.322124,335000.0,360274.795532
6,305836.124503,295181.095398,295851.213211,312500.0,301269.557672
7,208576.035479,209884.220547,209235.054099,227000.0,207391.383098
8,151338.890860,165166.752543,160364.541943,127000.0,154590.000044
9,117548.202657,114976.747281,113038.030787,140000.0,123345.549486


In [283]:
sub_dataframe= pd.read_csv("../data/test_dummied.csv")
sub_dataframe=sub_dataframe.set_index('Id')
X_sub=np.array(sub_dataframe)

submission_dataset=pd.DataFrame({'lasso':lasso.predict(X_sub),
                                'ridge':ridge.predict(X_sub),
                                'ENST':ENST.predict(X_sub)})



ValueError: Input contains NaN, infinity or a value too large for dtype('float64').

In [282]:
sub_dataframe

,1stFlrSF,2ndFlrSF,3SsnPorch,BedroomAbvGr,BsmtFinSF1,BsmtFinSF2,BsmtFullBath,BsmtHalfBath,BsmtUnfSF,EnclosedPorch,...,SaleType_CWD,SaleType_Con,SaleType_ConLD,SaleType_ConLI,SaleType_ConLw,SaleType_New,SaleType_Oth,SaleType_WD,Street_Grvl,Street_Pave
Id,,,,,,,,,,,,,,,,,,,,,
1461,-0.657390,-0.786163,-0.099206,-1.040209,0.108207,0.634970,-0.795568,-0.243216,-0.691967,-0.386120,...,0,0,0,0,0,0,0,1,0,1
1462,0.483570,-0.786163,-0.099206,0.190439,1.136688,-0.290049,-0.795568,-0.243216,-0.385662,-0.386120,...,0,0,0,0,0,0,0,1,0,1
1463,-0.573069,0.892848,-0.099206,0.190439,0.838316,-0.290049,-0.795568,-0.243216,-0.991516,-0.386120,...,0,0,0,0,0,0,0,1,0,1
1464,-0.578339,0.837759,-0.099206,0.190439,0.411100,-0.290049,-0.795568,-0.243216,-0.570346,-0.386120,...,0,0,0,0,0,0,0,1,0,1
1465,0.354455,-0.786163,-0.099206,-1.040209,-0.355175,-0.290049,-0.795568,-0.243216,0.990459,-0.386120,...,0,0,0,0,0,0,0,1,0,1
1466,-1.007846,1.350325,-0.099206,0.190439,-0.949659,-0.290049,-0.795568,-0.243216,0.418389,-0.386120,...,0,0,0,0,0,0,0,1,0,1
1468,-0.939336,0.832969,-0.099206,0.190439,-0.949659,-0.290049,-0.795568,-0.243216,0.476947,-0.386120,...,0,0,0,0,0,0,0,1,0,1
1469,0.515191,-0.786163,-0.099206,-1.040209,0.490214,-0.290049,1.128931,-0.243216,0.193165,-0.386120,...,0,0,0,0,0,0,0,1,0,1
1470,-0.694280,-0.786163,-0.099206,-1.040209,0.867701,0.211003,1.128931,-0.243216,-1.300073,-0.386120,...,0,0,0,0,0,0,0,1,0,1


## 4.2 Generate Test Design
### Outputs:
- Test Design

## 4.3 Build Model
### Outputs:
- Parameter Settings
- Models
- Model Descriptions

## 4.4 Assess Model
### Outputs:

- Model Assessment
- Revised Parameter
- Settings